## Project 2 - Housing Project

### Kaggle Submission

This notebook will replicate all the cleaning, processing and other necessary steps done in notebook 1. However, now we will be working on the "text.csv".

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

C:\Users\tiffa\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [26]:
test = pd.read_csv("./datasets/test.csv", low_memory = False)
hdb_cpi = pd.read_csv("./datasets/hdb_cpi.csv", skiprows = 9) 

In [27]:
print(test.shape)
print(test.dtypes)

(16737, 76)
id                     int64
Tranc_YearMonth       object
town                  object
flat_type             object
block                 object
                      ...   
sec_sch_name          object
cutoff_point           int64
affiliation            int64
sec_sch_latitude     float64
sec_sch_longitude    float64
Length: 76, dtype: object


In [30]:
# Repeat the same cleaning processes
hdb_cpi = hdb_cpi.head(130)
hdb_cpi[['year', 'quarter']] = hdb_cpi['Data Series'].str.split(expand=True)
hdb_cpi['year'] = pd.to_numeric(hdb_cpi['year'])
hdb_cpi['quarter'] = hdb_cpi['Data Series'].str.extract(r'(\d+)Q').astype(int)
hdb_cpi.head()

,Data Series,Total (Index),year,quarter
0,2023 2Q,176.2,2023,2
1,2023 1Q,173.6,2023,1
2,2022 4Q,171.9,2022,4
3,2022 3Q,168.1,2022,3
4,2022 2Q,163.9,2022,2


In [31]:
# Add quarter to data set for merging of cpi
test['date'] = pd.to_datetime(test['Tranc_YearMonth'])
test['quarter'] = test['date'].dt.quarter

In [32]:
# merge HDB resale prices dataset with cpi and computing adjusted prices with price index (Q1 2009 = 100)

test_merged = test.merge(hdb_cpi, left_on=['Tranc_Year', 'quarter'],
                      right_on=['year', 'quarter'], how='left')
test_merged = test_merged.drop(columns = ["date", "Data Series"])


In [ ]:
test.head()

In [33]:
# convert columns with "Y" or "N" to "1" and "0"

test['commercial'] = test['commercial'].apply(lambda x:1 if x == "Y" else 0)
test['market_hawker'] = test['market_hawker'].apply(lambda x:1 if x == "Y" else 0)
test['multistorey_carpark'] = test['multistorey_carpark'].apply(lambda x:1 if x == "Y" else 0)
test['precinct_pavilion'] = test['precinct_pavilion'].apply(lambda x:1 if x == "Y" else 0)

In [34]:
# Read through data dictionary and manually remove redundant variables by dropping columns. Explanation in appendix

# columns to drop
drop_col = ['block', 'street_name','floor_area_sqft','Mall_Within_500m', 'Mall_Within_1km',
       'Mall_Within_2km','full_flat_type',  'Hawker_Within_500m', 'residential','multistorey_carpark',
       'Hawker_Within_1km', 'Hawker_Within_2km','1room_sold', '2room_sold',
       '3room_sold', '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental','pri_sch_latitude', 'bus_interchange',
       'pri_sch_longitude', 'sec_sch_nearest_dist', 'sec_sch_name',
       'cutoff_point', 'affiliation', 'sec_sch_latitude','bus_stop_name','postal','mid_storey','lower','upper', 'sec_sch_longitude','bus_stop_longitude','bus_stop_latitude','vacancy','storey_range']

test.drop(drop_col, axis = 1, inplace=True)

In [35]:
test.isna().sum().sort_values(ascending = False)

Mall_Nearest_Distance        84
id                            0
Tranc_YearMonth               0
planning_area                 0
Hawker_Nearest_Distance       0
hawker_food_stalls            0
hawker_market_stalls          0
mrt_nearest_distance          0
mrt_name                      0
mrt_interchange               0
mrt_latitude                  0
mrt_longitude                 0
bus_stop_nearest_distance     0
pri_sch_nearest_distance      0
pri_sch_name                  0
pri_sch_affiliation           0
date                          0
Longitude                     0
Latitude                      0
total_dwelling_units          0
Tranc_Month                   0
town                          0
flat_type                     0
floor_area_sqm                0
flat_model                    0
lease_commence_date           0
Tranc_Year                    0
mid                           0
precinct_pavilion             0
address                       0
hdb_age                       0
max_floo

In [36]:
median = test.Mall_Nearest_Distance.median
test['Mall_Nearest_Distance']=test['Mall_Nearest_Distance'].fillna(median)
test['Mall_Nearest_Distance'] = pd.to_numeric(test['Mall_Nearest_Distance'], errors='coerce')

# Check for NaN values
nan_mask = test['Mall_Nearest_Distance'].isna()

# Check for Inf values (positive and negative infinity)
inf_mask = np.isinf(test['Mall_Nearest_Distance'])

# Replace NaN values with 0 in the 'Mall_Nearest_Distance' column
test['Mall_Nearest_Distance'].fillna(0, inplace=True)

In [37]:
test.shape

(16737, 36)

In [38]:
# combine flat_types with 1 or 2 ROOMs to the same column
test['flat_type'] = test['flat_type'].apply(lambda x: '1-2 ROOM' if x in ('1 ROOM', '2 ROOM') else x)

In [39]:
# convert to numeric terms
for x in ['floor_area_sqm','mid']:
    test[x] = pd.to_numeric(test[x])

In [40]:
# map towns according to region

town_mapping = {'KALLANG/WHAMPOA':'RCR' , 
                'BISHAN':'RCR', 
                'BUKIT BATOK':'OCR', 
                'YISHUN':'OCR', 
                'GEYLANG':'RCR',
               'HOUGANG':'OCR', 
                'BEDOK':'OCR', 
                'SENGKANG':'OCR',
                'TAMPINES':'OCR', 
                'SERANGOON':'RCR',
               'BUKIT MERAH':'CCR', 
                'BUKIT PANJANG':'OCR', 
                'WOODLANDS':'OCR',
                'JURONG WEST':'OCR',
                'TOA PAYOH':'RCR', 
                'CHOA CHU KANG':'OCR', 
                'SEMBAWANG':'OCR',
                'ANG MO KIO':'OCR',
               'PASIR RIS':'OCR', 
                'CLEMENTI':'OCR', 
                'PUNGGOL':'OCR', 
                'JURONG EAST':'OCR',
                'CENTRAL AREA':'CCR',
                'QUEENSTOWN':'CCR', 
                'BUKIT TIMAH':'CCR', 
                'MARINE PARADE':'OCR'}
test['town_region'] = test['town'].map(town_mapping)


In [41]:
# remaining lease
test['remaining_lease'] = 99-(test['Tranc_Year']- test['lease_commence_date'])

In [44]:
# top pri sch

top_pri_schools = ['Ai Tong School',
'Anglo-Chinese School (Junior)',
'Anglo-Chinese School (Primary)*',
'Catholic High School',
'CHIJ Saint Nicholas Girls\' School',
'Nan Hua Primary School',
'Nanyang Primary School',
'Raffles Girls’ Primary School',
'Rosyth School', 'Henry Park Primary School','Saint Joseph\'s Institution Junior'
'Saint Hilda\'s Primary School',
'Tao Nan School',
'Maha Bodhi School',
'Pei Hwa Presbyterian Primary School']

test['top_pri_sch'] = test.pri_sch_name.isin(top_pri_schools).astype(int)

# drop pri sch name column as not needed anymore
test = test.drop(['pri_sch_name'], axis=1)


In [45]:
test.shape

(16737, 38)

In [46]:
test.columns

Index(['id', 'Tranc_YearMonth', 'town', 'flat_type', 'floor_area_sqm',
       'flat_model', 'lease_commence_date', 'Tranc_Year', 'Tranc_Month', 'mid',
       'address', 'hdb_age', 'max_floor_lvl', 'year_completed', 'commercial',
       'market_hawker', 'precinct_pavilion', 'total_dwelling_units',
       'Latitude', 'Longitude', 'planning_area', 'Mall_Nearest_Distance',
       'Hawker_Nearest_Distance', 'hawker_food_stalls', 'hawker_market_stalls',
       'mrt_nearest_distance', 'mrt_name', 'mrt_interchange', 'mrt_latitude',
       'mrt_longitude', 'bus_stop_nearest_distance',
       'pri_sch_nearest_distance', 'pri_sch_affiliation', 'date', 'quarter',
       'town_region', 'remaining_lease', 'top_pri_sch'],
      dtype='object')

In [47]:
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['town_region', 'flat_type', 'flat_model','town', 'mrt_name']
encoder = OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore')
encoded_features = encoder.fit_transform(test[columns_to_encode])
encoded_column_names = encoder.get_feature_names_out(input_features=columns_to_encode) 
encoded_df = pd.DataFrame(encoded_features, columns=encoded_column_names)
test = test.drop(columns=columns_to_encode) 
test = pd.concat([test, encoded_df], axis=1) 

C:\Users\tiffa\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
# create bins for remaining lease based on summary stats of test data (as above)
bins = [45,66,74,84,98]
bins_lease = pd.cut(test['remaining_lease'],bins)
lease_dummy = pd.get_dummies(bins_lease, prefix='lease', dtype = 'int', drop_first=True)
test1 = pd.merge(test, lease_dummy, left_index=True, right_index=True)
test1 = test1.drop(columns="remaining_lease")

In [49]:
# create bins based on summary stats 
bins = [49,80,96,113,280]
bins_floor_area = pd.cut(test1['floor_area_sqm'],bins)
floor_dummy = pd.get_dummies(bins_floor_area, prefix='floor_area_', dtype = 'int', drop_first=True)
test2 = pd.merge(test1, floor_dummy, left_index=True, right_index=True)
test2 = test2.drop(columns="floor_area_sqm")

In [51]:
test_cleaned = test2[['mid',
 'max_floor_lvl',
 'commercial',
 'market_hawker',
 'precinct_pavilion',
 'total_dwelling_units',
 'Mall_Nearest_Distance',
 'Hawker_Nearest_Distance',
 'hawker_food_stalls',
 'hawker_market_stalls',
 'mrt_nearest_distance',
 'bus_stop_nearest_distance',
 'pri_sch_nearest_distance',
 'pri_sch_affiliation',
 'top_pri_sch',
 'town_region_OCR',
 'flat_type_1-2 ROOM',
 'flat_type_3 ROOM',
 'flat_type_4 ROOM',
 'flat_type_5 ROOM',
 'flat_model_Apartment',
 'flat_model_DBSS',
 'flat_model_Improved-Maisonette',
 'flat_model_Maisonette',
 'flat_model_Model A-Maisonette',
 'flat_model_Model A2',
 'flat_model_New Generation',
 'flat_model_Premium Apartment',
 'flat_model_Premium Apartment Loft',
 'flat_model_Premium Maisonette',
 'flat_model_Simplified',
 'flat_model_Standard',
 'flat_model_Terrace',
 'flat_model_Type S1',
 'flat_model_Type S2',
 'town_BEDOK',
 'town_BUKIT BATOK',
 'town_BUKIT MERAH',
 'town_CENTRAL AREA',
 'town_CLEMENTI',
 'town_HOUGANG',
 'town_JURONG EAST',
 'town_MARINE PARADE',
 'town_PUNGGOL',
 'town_QUEENSTOWN',
 'town_SENGKANG',
 'mrt_name_Aljunied',
 'mrt_name_Ang Mo Kio',
 'mrt_name_Bartley',
 'mrt_name_Bedok',
 'mrt_name_Bedok North',
 'mrt_name_Bedok Reservoir',
 'mrt_name_Bencoolen',
 'mrt_name_Bendemeer',
 'mrt_name_Boon Keng',
 'mrt_name_Braddell',
 'mrt_name_Bras Basah',
 'mrt_name_Buangkok',
 'mrt_name_Bukit Batok',
 'mrt_name_Bukit Gombak',
 'mrt_name_Caldecott',
 'mrt_name_Chinese Garden',
 'mrt_name_Clarke Quay',
 'mrt_name_Clementi',
 'mrt_name_Commonwealth',
 'mrt_name_Dakota',
 'mrt_name_Dover',
 'mrt_name_Eunos',
 'mrt_name_Farrer Park',
 'mrt_name_Geylang Bahru',
 'mrt_name_Hillview',
 'mrt_name_Holland Village',
 'mrt_name_Hougang',
 'mrt_name_Jalan Besar',
 'mrt_name_Kaki Bukit',
 'mrt_name_Kallang',
 'mrt_name_Kembangan',
 'mrt_name_Kovan',
 'mrt_name_Labrador Park',
 'mrt_name_Lavender',
 'mrt_name_Lorong Chuan',
 'mrt_name_Marymount',
 'mrt_name_Mattar',
 'mrt_name_Mountbatten',
 'mrt_name_Nicoll Highway',
 'mrt_name_Novena',
 'mrt_name_Potong Pasir',
 'mrt_name_Queenstown',
 'mrt_name_Redhill',
 'mrt_name_Rochor',
 'mrt_name_Tai Seng',
 'mrt_name_Tanjong Pagar',
 'mrt_name_Telok Blangah',
 'mrt_name_Tiong Bahru',
 'mrt_name_Toa Payoh',
 'mrt_name_Ubi',
 'mrt_name_Woodleigh',
 'mrt_name_Yio Chu Kang',
 'lease_(66, 74]',
 'lease_(74, 84]',
 'lease_(84, 98]',
 'floor_area__(80, 96]',
 'floor_area__(96, 113]',
 'floor_area__(113, 280]']]

In [52]:
test_cleaned.shape

(16737, 104)

In [53]:
test_cleaned.shape

(16737, 104)

In [54]:
test_cleaned.to_csv("./datasets/test_cleaned.csv")